In [ ]:
!nvidia-smi

Thu Oct 19 03:18:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip -q install langchain[llms] openai pypdf tiktoken PyPDF2 transformers accelerate bitsandbytes torch sentencepiece

In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate
from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import json
import textwrap
import PyPDF2
import re

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
#                                              device_map='auto',
#                                              torch_dtype=torch.float16,
#                                              load_in_4bit=True,
#                                              bnb_4bit_quant_type="nf4",
#                                              bnb_4bit_compute_dtype=torch.float16)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-13b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-13b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)

In [ ]:
# Unquantized Model
# tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-13b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-13b-chat-hf",
#                                              device_map='auto',
#                                              torch_dtype="auto")

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 750,
                do_sample=True,
                top_k=25,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an AI (PROFESSSION DOCTOR,etc), Carefully analyze this part of XYZ document. Summarize the Following for me:"""


In [ ]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=750,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [ ]:


def clean_extra_whitespace(text):
    """
    Removes extra whitespaces from the text.
    """
    text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with a single space
    return text

def clean_non_ascii_chars(text):
    """
    Removes non-ASCII characters from the text.
    """
    return ''.join(char for char in text if ord(char) < 128)

def group_broken_paragraphs(text):
    """
    Groups broken paragraphs. Assumes that a paragraph is broken if it ends with a hyphen followed immediately by a word.
    """
    return re.sub(r'-\s*([^ \n]+)', r'\1', text)

def clean_text(text):
    """
    Combined function to clean the text.
    """
    text = clean_extra_whitespace(text)
    text = clean_non_ascii_chars(text)
    text = group_broken_paragraphs(text)
    return text


In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.01,'max_length': 750, 'top_k' :25})

In [ ]:
default_template = """
              You are Helpful Assistant, Carefully analyze the given chunk of document.
              Write a summary of the following text delimited by triple backticks.
              Dont Loose Important Information and Provide All Relevant, Accurate Information.
              Return your response which covers all key points of the chunk.
              ```{text}```
              SUMMARY:
              """


simple_system_prompt="You are Helpful legal Assistant, Master at Summarizing Docs, Carefully analyze the provided chunk of document."
simple_instruction="""Write the best summary of the following chunk Given Below :
              -------------------------------
              ```{text}```
              -------------------------------

              Write a concise Summary which is As Accurate as Possible and Covers all aspects of the this Document .
              Include other Relevant Information
              ONLY ANSWER IN BULLET POINTS:"""

simple_template = get_prompt(simple_instruction, simple_system_prompt)
print(simple_template)

In [ ]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.docstore.document import Document
import torch

def process_directory_with_stuff_chain(root_dir, llm):
    """
    Traverse the specified directory, accessing each sub-directory and their text files.
    For each text file, clean its content, split it into chunks,
    summarize each chunk using the StuffDocumentsChain, and then save the summarized content to a new file.
    """

# Summarize the chunks
    # system_prompt = "You are Helpful legal Assistant, Carefully analyze this chunk of legal/Law document: "
    # instruction = "Please extract the most important and relevant information from the following legal text chunk, ensuring to capture all important information. Prioritize coherence and consistency, avoiding redundancy. Ensure the information is clear and can be understood in the context of a larger legal document : {text}"
    # advanced_template = get_prompt(instruction, system_prompt)
    # #print(advanced_template)

    prompt = PromptTemplate.from_template(simple_template)

    # Define the LLMChain
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # Define the StuffDocumentsChain
    stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

    for subdir, _, files in os.walk(root_dir):
        print(f"Processing directory: {subdir}")
        for file in files:
            if file.endswith('.txt') and not file.endswith('_cleaned.txt') and not file.endswith('_summary.txt'):
                file_path = os.path.join(subdir, file)
                print(f"Processing file: {file_path}")

                with open(file_path, 'r', encoding='utf-8') as infile:
                    content = infile.read()
                if not content:
                    print(f"File {file_path} is empty!")
                    continue

                cleaned_content = clean_text(content)
                text_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=250)
                texts = text_splitter.split_text(cleaned_content)
                docs = [Document(page_content=t) for t in texts]
                all_summaries = []
                common_summary = ""
                for i, doc in enumerate(docs):
                    print(f"Processing chunk {i + 1} of {len(docs)}")
                    summarized_doc = stuff_chain.run([doc])
                    print("First few chars of summary:", summarized_doc[:100])
                    if not summarized_doc:
                        print(f"No summary generated for chunk {i + 1}")
                        continue
                    Chunk_Number=f"Summary of Chunk Number {i + 1} of {len(docs)} : "
                    all_summaries.append(Chunk_Number)
                    all_summaries.append(summarized_doc)
                    common_summary += " " + summarized_doc
                    torch.cuda.empty_cache()
                #print("COMMON SUMMARY ",common_summary)
                #print("COMMON SUMMARY OVER -----------------------",)
                if not all_summaries:
                    print(f"No summaries generated for file {file_path}")
                    continue
                #print("ALL SUMMARY",all_summaries)
                #print("ALL SUMMARY OVER -----------------------",)
                summary_file_path = os.path.join(subdir, file.split('.txt')[0] + '_summary.txt')
                with open(summary_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.write('\n'.join(all_summaries))
                    print(f"Summaries saved to: {summary_file_path}")



START MASS SUMMARIZATION

In [ ]:
# Define the base directory path
base_path = "/content/101_110"
zip_path = base_path + ".zip"

# Call the processing function
#process_directory_summarize("/content/test", llm)
process_directory_with_stuff_chain(base_path, llm)


In [ ]:
from google.colab import files
import shutil
# Create a zip file
shutil.make_archive(base_path, 'zip', base_path)

# Download the zip file using Google Colab's utility
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>